<a href="https://colab.research.google.com/github/karensheng/API-use-case-demo/blob/main/Youtube_search_demo_Aisa_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Load API key from environment variable
API_KEY = os.getenv("AISA_API_KEY", "<YOUR AISA API KEY>")

In [2]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

# Base API URL
base_url = "https://api.aisa.one/apis/v1/youtube/search"

# Initial query parameters
params = {
    "engine": "youtube",
    "q": "humanoid robotics",
    "sp": "sp",
    "gl": "us"
}

headers = {"Authorization": f"Bearer {API_KEY}"}

all_collected_videos = []
next_page_token = None
MAX_RESULTS = 200

while len(all_collected_videos) < MAX_RESULTS:
    # Add next_page_token to params if available
    if next_page_token:
        params["next_page_token"] = next_page_token
    elif "next_page_token" in params:
        del params["next_page_token"]

    response = requests.get(base_url, params=params, headers=headers)
    response_data = json.loads(response.text)

    # Extract videos from the main 'videos' list
    if 'videos' in response_data:
        for video in response_data['videos']:
            if len(all_collected_videos) >= MAX_RESULTS:
                break
            all_collected_videos.append({
                'id': video.get('id'),
                'title': video.get('title'),
                'link': video.get('link'),
                'type': 'video',
                'views': video.get('views'),
                'published_time': video.get('published_time')
            })

    # Extract videos from 'sections' if available
    if 'sections' in response_data:
        for section in response_data['sections']:
            if 'items' in section:
                for item in section['items']:
                    if len(all_collected_videos) >= MAX_RESULTS:
                        break
                    if item.get('type') == 'video' or 'videos' in item.get('link', '') or 'youtube.com/watch' in item.get('link', '') or 'youtube.com/shorts' in item.get('link', ''):
                         all_collected_videos.append({
                            'id': item.get('id'),
                            'title': item.get('title'),
                            'link': item.get('link'),
                            'type': item.get('type', 'video'),
                            'views': item.get('views'),
                            'published_time': item.get('published_time')
                        })

    # Extract shorts from 'shorts' sections
    if 'shorts' in response_data:
        for shorts_section in response_data['shorts']:
            if 'items' in shorts_section:
                for short_item in shorts_section['items']:
                    if len(all_collected_videos) >= MAX_RESULTS:
                        break
                    all_collected_videos.append({
                        'id': short_item.get('id'),
                        'title': short_item.get('title'),
                        'link': short_item.get('link'),
                        'type': 'short',
                        'views': short_item.get('views'),
                        'published_time': short_item.get('published_time')
                    })


    # Get the next page token
    next_page_token = response_data.get('pagination', {}).get('next_page_token')

    # If no more pages, break the loop
    if not next_page_token:
        break

# Trim to MAX_RESULTS if more were collected
final_results = all_collected_videos[:MAX_RESULTS]

df_paginated_results = pd.DataFrame(final_results)

# Filter out videos with less than 5000 views locally
df_paginated_results = df_paginated_results[df_paginated_results['views'].fillna(0).astype(int) >= 5000]

# Remove duplicates by 'id', keeping the first occurrence
df_paginated_results = df_paginated_results.drop_duplicates(subset=['id']).reset_index(drop=True)

# Sort the DataFrame by 'views' in descending order
df_paginated_results = df_paginated_results.sort_values(by='views', ascending=False).reset_index(drop=True)

# Function to parse relative time strings to datetime objects
def parse_published_time(time_str):
    if pd.isna(time_str) or not isinstance(time_str, str):
        return pd.NaT

    time_str = time_str.lower().strip()
    current_time = datetime.now()

    if 'ago' in time_str:
        parts = time_str.split()
        if len(parts) >= 2:
            value = int(parts[0])
            unit = parts[1]

            if 'second' in unit:
                return current_time - timedelta(seconds=value)
            elif 'minute' in unit:
                return current_time - timedelta(minutes=value)
            elif 'hour' in unit:
                return current_time - timedelta(hours=value)
            elif 'day' in unit:
                return current_time - timedelta(days=value)
            elif 'week' in unit:
                return current_time - timedelta(weeks=value)
            elif 'month' in unit:
                return current_time - timedelta(days=value * 30) # Approximation
            elif 'year' in unit:
                return current_time - timedelta(days=value * 365) # Approximation
    return pd.NaT

# Apply the parsing function and filter for the last 12 months
df_paginated_results['published_time_dt'] = df_paginated_results['published_time'].apply(parse_published_time)

one_year_ago = datetime.now() - timedelta(days=365)
df_paginated_results = df_paginated_results[df_paginated_results['published_time_dt'] >= one_year_ago].reset_index(drop=True)


print(f"Total items collected: {len(df_paginated_results)}")
display(df_paginated_results.head(10))

Total items collected: 32


,id,title,link,type,views,published_time,published_time_dt
0,xbvkdPhfzwU,2025 Robot Expo Quadruped Robot Driving Test. ...,https://www.youtube.com/watch?v=xbvkdPhfzwU,video,33663427,3 months ago,2025-10-09 22:37:30.966120
1,I44_zbEwz_w,"Walk, Run, Crawl, RL Fun | Boston Dynamics | A...",https://www.youtube.com/watch?v=I44_zbEwz_w,video,8725024,9 months ago,2025-04-12 22:37:30.966139
2,j31dmodZ-5c,The Problem with this Humanoid Robot,https://www.youtube.com/watch?v=j31dmodZ-5c,video,6267700,2 months ago,2025-11-08 22:37:30.966145
3,2HQ84TVcbMw,"Meet the Realbotix Robots: Less Robot, More Human",https://www.youtube.com/watch?v=2HQ84TVcbMw,video,2122792,11 months ago,2025-02-11 22:37:30.966156
4,f3c4mQty_so,I Tried the First Humanoid Home Robot. It Got ...,https://www.youtube.com/watch?v=f3c4mQty_so,video,2115501,2 months ago,2025-11-08 22:37:30.966160
5,CbHeh7qwils,Progress made on AI-powered humanoid robots | ...,https://www.youtube.com/watch?v=CbHeh7qwils,video,1069092,2 days ago,2026-01-05 22:37:30.966163
6,EdIAA1zROiA,NEO Gamma Humanoid Robot Vacuums at Nvidia GTC...,https://www.youtube.com/watch?v=EdIAA1zROiA,video,755347,9 months ago,2025-04-12 22:37:30.966170
7,pvOLxYhpwO0,China Just Dropped Self-Evolving AI Robots Wit...,https://www.youtube.com/watch?v=pvOLxYhpwO0,video,575138,2 months ago,2025-11-08 22:37:30.966176
8,zii2FiFBl5k,China's New AI Robot Just Broke a Human Skill ...,https://www.youtube.com/watch?v=zii2FiFBl5k,video,449523,12 days ago,2025-12-26 22:37:30.966180
9,fBFwFyZbewo,Top 10 NEW Humanoid Robots of 2025 (Updated),https://www.youtube.com/watch?v=fBFwFyZbewo,video,432523,10 months ago,2025-03-13 22:37:30.966183


In [3]:
df_videos = df_paginated_results[df_paginated_results['type'] == 'video'].reset_index(drop=True)
df_shorts = df_paginated_results[df_paginated_results['type'] == 'short'].reset_index(drop=True)

# Sort df_videos by published_time_dt in descending order
df_videos = df_videos.sort_values(by='published_time_dt', ascending=False).reset_index(drop=True)

# Sort df_shorts by published_time_dt in descending order
df_shorts = df_shorts.sort_values(by='published_time_dt', ascending=False).reset_index(drop=True)

print("Videos DataFrame (df_videos):")
display(df_videos.head(10))

print("\nShorts DataFrame (df_shorts):")
display(df_shorts.head(10))

Videos DataFrame (df_videos):


,id,title,link,type,views,published_time,published_time_dt
0,rR8OPn3if2U,Dance Battling a $60K Humanoid Robot,https://www.youtube.com/watch?v=rR8OPn3if2U,video,5702,9 hours ago,2026-01-07 13:37:30.966246
1,T4ekeXQ2B6Q,Unitree New Robot Goes Superhuman on Camera (S...,https://www.youtube.com/watch?v=T4ekeXQ2B6Q,video,46166,23 hours ago,2026-01-06 23:37:30.966207
2,R2sBaJmxYic,Hyundai reveals humanoid robot Atlas and danci...,https://www.youtube.com/watch?v=R2sBaJmxYic,video,8342,1 day ago,2026-01-06 22:37:30.966243
3,FhzPcwV3_CA,A Humanoid Robot for Your Desk (Featuring Wall...,https://www.youtube.com/watch?v=FhzPcwV3_CA,video,18769,1 day ago,2026-01-06 22:37:30.966236
4,e73kf_iLAP0,New Atlas Humanoid Revealed! Boston Dynamics &...,https://www.youtube.com/watch?v=e73kf_iLAP0,video,211358,1 day ago,2026-01-06 22:37:30.966194
5,sd8ivhpjI6g,The Beginning of Your Tomorrow I Boston Dynamics,https://www.youtube.com/watch?v=sd8ivhpjI6g,video,333335,2 days ago,2026-01-05 22:37:30.966191
6,n6ISdRkS37I,How Boston Dynamics upgraded the Atlas robot,https://www.youtube.com/watch?v=n6ISdRkS37I,video,400223,2 days ago,2026-01-05 22:37:30.966186
7,CbHeh7qwils,Progress made on AI-powered humanoid robots | ...,https://www.youtube.com/watch?v=CbHeh7qwils,video,1069092,2 days ago,2026-01-05 22:37:30.966163
8,VTASY2YSC_0,A Lonely AI Engineer in Berlin Lives With a Hu...,https://www.youtube.com/watch?v=VTASY2YSC_0,video,23548,3 days ago,2026-01-04 22:37:30.966234
9,jx1DHC8JLT4,Inside Tesla’s AI Robot Factory: Building the ...,https://www.youtube.com/watch?v=jx1DHC8JLT4,video,27368,3 days ago,2026-01-04 22:37:30.966224



Shorts DataFrame (df_shorts):


,id,title,link,type,views,published_time,published_time_dt
